In [1]:
import pandas as pd
import numpy as np
import glob

#### Making large dataset of population-weight ZIP code smoke estimates and population-weight temperature data
##### Also need to "melt" the dataframe so the Zips are separate rows instead of column headers

In [5]:
# Start with Kate O'Dell's product
# Read in all years of files for smoke and then total PM2.5
inputfiles = glob.glob('C:/Users/olivia.sablan/OneDrive - State of New Mexico/Documents/Data/smoke/KateZip/nm_smoke*.csv')
df_from_each_file = (pd.read_csv(f) for f in inputfiles)
smoke = pd.concat(df_from_each_file,sort=False)
meltsmoke= smoke.melt(id_vars=['Date'],var_name='Zip',value_name='smokepm25')

inputfiles = glob.glob('C:/Users/olivia.sablan/OneDrive - State of New Mexico/Documents/Data/smoke/KateZip/nm_total*.csv')
df_from_each_file = (pd.read_csv(f) for f in inputfiles)
total = pd.concat(df_from_each_file,sort=False)
melttotal= total.melt(id_vars=['Date'],var_name='Zip',value_name='totalpm25')

mergedZip= pd.merge(meltsmoke, melttotal, on = ['Date', 'Zip'])
mergedZip.to_csv('C:/Users/olivia.sablan/OneDrive - State of New Mexico/Documents/Data/smoke/KateZip/AllZipSmoke_Total_os.csv')


In [8]:
# Next, do the same for Bonne Ford's smoke product
# This product is only for 2022 so we don't need to "glob" to read it a bunch of files
smoke = pd.read_csv("C:/Users/olivia.sablan/OneDrive - State of New Mexico/Documents/Data/smoke/BonneZip/nm_smoke_zipcode_2022_final.csv")
total = pd.read_csv("/Users/olivia.sablan/OneDrive - State of New Mexico/Documents/Data/smoke/BonneZip/nm_totalpm_zipcode_2022_final.csv")
melttotal = total.melt(id_vars = ['Date'], var_name = 'Zip', value_name= 'totalpm25')
meltsmoke= smoke.melt(id_vars = ['Date'], var_name = 'Zip', value_name= 'smokepm25')
alltogether = pd.merge(meltsmoke, melttotal, on = ['Date', 'Zip'])
alltogether.to_csv('/Users/olivia.sablan/OneDrive - State of New Mexico/Documents/Data/smoke/BonneZip/bigPurpleAirZip_os.csv')

In [4]:
# Repeat for Bonne Ford's other smoke product that uses Kate O'Dell's variogram parameters (vgp) for kriging
smoke = pd.read_csv("C:/Users/olivia.sablan/OneDrive - State of New Mexico/Documents/Data/smoke/BonneZip/nm_smoke_zipcode_2022_final_KOvgp.csv")
total = pd.read_csv("/Users/olivia.sablan/OneDrive - State of New Mexico/Documents/Data/smoke/BonneZip/nm_totalpm_zipcode_2022_final_KOvgp.csv")
melttotal = total.melt(id_vars = ['Date'], var_name = 'Zip', value_name= 'totalpm25')
meltsmoke= smoke.melt(id_vars = ['Date'], var_name = 'Zip', value_name= 'smokepm25')
alltogether = pd.merge(meltsmoke, melttotal, on = ['Date', 'Zip'])
alltogether.to_csv('/Users/olivia.sablan/OneDrive - State of New Mexico/Documents/Data/smoke/BonneZip/bigPurpleAirZip_os_KOvgp.csv')

In [8]:
# Also, repeated for Kamal's smoke product
smoke = pd.read_csv("C:/Users/olivia.sablan/OneDrive - State of New Mexico/Documents/Data/smoke/KamalZip/BI_PM25_TOT_AVG_FUSED_zipcode_new.csv")
smokemax = pd.read_csv("/Users/olivia.sablan/OneDrive - State of New Mexico/Documents/Data/smoke/KamalZip/BI_PM25_TOT_MAX_FUSED_zipcode_new.csv")
total = pd.read_csv("/Users/olivia.sablan/OneDrive - State of New Mexico/Documents/Data/smoke/KamalZip/Daily_PM25_TOT_AVG_FUSED_zipcode_new.csv")
totalmax = pd.read_csv("/Users/olivia.sablan/OneDrive - State of New Mexico/Documents/Data/smoke/KamalZip/Daily_PM25_TOT_MAX_FUSED_zipcode_new.csv")

meltsmoke = smoke.melt(id_vars = ['Date'], var_name = 'Zip', value_name= 'smokepm25')
meltsmokemax = smokemax.melt(id_vars = ['Date'], var_name = 'Zip', value_name= 'maxsmokepm25')
melttotal = total.melt(id_vars = ['Date'], var_name = 'Zip', value_name= 'totalpm25')
melttotalmax= totalmax.melt(id_vars = ['Date'], var_name = 'Zip', value_name= 'maxtotalpm25')

merge1 = pd.merge(meltsmoke, meltsmokemax, on = ['Date', 'Zip'])
merge2 = pd.merge(merge1, melttotal, on = ['Date', 'Zip'])
alltogether = pd.merge(merge2, melttotalmax, on = ['Date', 'Zip'])

alltogether.to_csv('/Users/olivia.sablan/OneDrive - State of New Mexico/Documents/Data/smoke/KamalZip/bigKamalZip_os.csv')

merge3 = pd.merge(meltsmoke, melttotal , on = ['Date', 'Zip'])
merge4 = pd.merge(meltsmokemax, melttotalmax , on = ['Date', 'Zip'])

merge3.to_csv('../Data/smoke/KamalZip/Kamal_average_reformat_all_OS.csv')
merge4 = merge4.rename(columns = {'maxsmokepm25': 'smokepm25'})
merge4.to_csv('../Data/smoke/KamalZip/Kamal_max_reformat_all_OS.csv')

In [82]:
# Read in population-weight temp data from Florida State University
poptemp = pd.read_csv('../Data/temp/max_temperature_zipcode_pop_weighted.csv')
# The dates are formatted in days since the unix date (Jan 1 1970), but we know this data is from Jan 1, 2016 to Dece 31 2022 so we can make a list of those dates
range = pd.date_range(start='1/1/2016', end='12/31/2022')
# Rename the data column headers in this usable format
newdates_temp = poptemp.iloc[:,1:].set_axis(range, axis=1)
# Add back in the ZIP codes
newdates_temp['Zip'] = poptemp['zipcode']
# Melt the data so that columns are "Date", "Zip", and "maxpoptemp"
melttemp = newdates_temp.melt(id_vars = ['Zip'], var_name = 'Date', value_name= 'maxpoptemp')
# Reformat date to make pretty
melttemp['Date'] = pd.to_datetime(melttemp['Date']).dt.date
# Save dataframe
melttemp.to_csv('../Data/temp/MaxPop_temp_OS.csv')

In [13]:
# Read in population-weight heat index data from Florida State University
poptemp = pd.read_csv('../Data/temp/max_HI_zipcode_pop_weighted.csv')
# The dates are formatted in days since the unix date (Jan 1 1970), but we know this data is from Jan 1, 2016 to Dece 31 2022 so we can make a list of those dates
range = pd.date_range(start='1/1/2016', end='12/31/2022')
# Rename the data column headers in this usable format
newdates_temp = poptemp.iloc[:,1:].set_axis(range, axis=1)
# Add back in the ZIP codes
newdates_temp['Zip'] = poptemp['zipcode']
# Melt the data so that columns are "Date", "Zip", and "maxpoptemp"
melttemp = newdates_temp.melt(id_vars = ['Zip'], var_name = 'Date', value_name= 'maxpopHI')
# Reformat date to make pretty
melttemp['Date'] = pd.to_datetime(melttemp['Date']).dt.date
# Save dataframe
melttemp.to_csv('../Data/temp/MaxPop_HI_OS.csv')